### 把1850——1875年的报纸报道中关于negro等的具体句子提取出来

In [2]:
import spacy
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
# 加载预训练的 spaCy 模型
nlp = spacy.load("en_core_web_sm")

In [19]:
# 处理一篇报道的代码
def contains_negro(text):
    doc = nlp(text)
    extracted_sentences = []
    target_words = ["blacks", "african", "africans", "negro", "negros", "negroes", 
                "negroid", "ethiopian", "ethiopians", "nigger", "niggers", "nigga", "niggas"]
    
    # 只提取包含列表词的句子
    for sent in doc.sents:
        if any(word in sent.text.lower() for word in target_words):
            extracted_sentences.append(sent.text)
    # 组合成大字符串
    combined_text = " ".join(extracted_sentences)
    return combined_text

In [9]:
# 打开数据
df = pd.read_csv(r"D:\American Stories\the civil war\data\black.news_geo.csv") 
df.head(3)

,Unnamed: 0,Article_id,Date,Page_number,Newspaper_name,Headline,Author,Article_body,seg_article_body,contains_keywords,fips,county,state
0,0,2_1850-01-01_p1_sn82007022_00393342894_1850010...,1850-01-01,p1,The Arkansas banner.,slavery Movement in ceoryia.,NaN,The House of Representatives of Georgia hay. p...,the house of representatives of georgia hay pa...,True,5119.0,pulaski,arkansas
1,1,24_1850-01-01_p1_sn82015378_00295872366_185001...,1850-01-01,p1,NaN,NaN,NaN,THE ABoEIrIoNrsTs.-Th Abolitioaisls afomuch mo...,the aboeirionrststhabolitioa isls afo much mov...,True,22071.0,orleans,louisiana
2,2,11_1850-01-01_p1_sn83030313_00271743014_185001...,1850-01-01,p1,NaN,NaN,NaN,"lb-Firs at Hempstsad, L. Ii four persons burne...",lbfirsathempstsadlii four persons burned to hh...,True,36061.0,new york,new york


In [10]:
# 删除“contains_keywords"列
df = df.drop('contains_keywords', axis=1)

In [20]:
# 在 Article_body 列上逐个应用 contains_negro 函数，显示 tqdm 进度条
tqdm.pandas()

df['filtered_text'] = df['Article_body'].progress_apply(contains_negro)

  0%|          | 0/270550 [00:00<?, ?it/s]

In [22]:
df.to_csv(r'D:\American Stories\the civil war\data\only_black_news.csv')

In [1]:
# list(df["filtered_text"])[:20]

### 提取出每个报道中关于黑人的形容词

In [1]:
from openai import OpenAI

client = OpenAI(
    api_key='你的api_key',
)
text = """
African Americans have faced systemic discrimination and are often described as resilient and strong in the face of adversity. The blacks in the community are known for their deep cultural heritage and warm hospitality.
"""

def chat_gpt(text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"""在{text}文本中，首先判断这个文本的内容是否有关于黑人群体的内容。其次，如果有关于黑人群体的内容 
                ,帮助我找出形容‘negro'的形容词或副词，不是’negro'本身。如果没有就返回‘无’ """
            }
        ]
    )
    return response.choices[0].message.content.strip()



chat_gpt(text)

'这个文本的内容确实有关于黑人群体的内容。形容“negro”的形容词或副词包括：\n\n1. systemic (systemic discrimination)\n2. resilient\n3. strong\n4. deep (deep cultural heritage)\n5. warm (warm hospitality)\n\n这些词汇都用来描述黑人群体的经历和文化特征。'